## Data

In [30]:
import os
import PIL
import path

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

In [31]:
sld_path = path.Path("../data")
#list_dir = os.listdir(sld_path/"Examples")

In [41]:
def normalize(loader):
    mean=0
    std=0
    nb_samples=0
    for data in loader:
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
        nb_samples += batch_samples

    mean /= nb_samples
    std /= nb_samples
    
    return transforms.Normalize(mean, std)

In [72]:
transform = transforms.Compose([
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Resize(size=(128,128)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [73]:
class sign_lang(Dataset):
    def __init__(self, path, transforms= None):
        self.transforms = transforms
        self.path = path
        
        self.data={cl:([(PIL.Image.open(path/cl/im),
                         int(cl)) for im in os.listdir(path/cl)]) 
                   for cl in os.listdir(path)}
        self.classes = self.data.keys()
        self.samp=[]
        for i in self.data:
            for e in self.data[i]:
                self.samp.append(e)
        self.targ = [self.samp[i][1] for i in range(len(self.samp))]
        
    def __len__(self):
        return len(self.samp)
    
    def targets(self):
        return self.targ
    
    def samples(self):
        return self.samp
    
    def __getitem__(self, index):
        sample = self.samp[index][0], self.samp[index][1]
        
        if self.transforms:    
            sample = self.transforms(sample[0]), sample[1]
        return sample
        
    
    def __repr__(self):
        return f"{__class__.__name__}:\nNum of datapoints: {len(self.samp)}\nRoot location: {self.path}"

In [74]:
def mkDataset():
    image_datasets = {x: sign_lang(sld_path/x, transform)
                      for x in ['train', 'valid', 'tests']}
    return image_datasets

In [75]:
def loadData():
    loaders = {x+'_loader': DataLoader(mkDataset()[x], batch_size=10,
                                              shuffle=True, num_workers=0)
                      for x in ['train', 'valid', 'tests']}
    return loaders

In [76]:
dl=loadData()

In [77]:
x, y = next(iter(dl["tests_loader"]))
x.shape

torch.Size([10, 3, 128, 128])

In [78]:
dataset_sizes = {x: len(mkDataset()[x]) for x in ['train', 'valid', 'tests']}
dataset_sizes

{'train': 1712, 'valid': 300, 'tests': 50}

In [11]:
#transforms.Normalize(mean, std)transforms.ToTensor()(PIL.Image.open(sld_path/"Examples/example_2.JPG"))

## Model

In [91]:
import torch.nn as nn
import torch.nn.functional as F
import time
import copy
import datetime

In [66]:
def input_layer():
    
    input_shape = next(iter(loadData()["train_loader"]))[0].shape
    return input_shape

In [86]:
class SignNet(nn.Module):
    def __init__(self, in_channels=input_layer()[1], out_channels=6):
        super(SignNet, self).__init__()
        
        self.Conv1 = nn.Conv2d(in_channels, out_channels, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.Conv2 = nn.Conv2d(out_channels, 16, 5)
        self.Conv3 = nn.Conv2d(16, 46, 5)
        self.fc1 = nn.Linear(46*12*12, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = self.pool(F.relu(self.Conv1(x)))
        x = self.pool(F.relu(self.Conv2(x)))
        x = self.pool(F.relu(self.Conv3(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
def softmax(x): return torch.exp(x)/torch.exp(x).sum(dim=1,keepdim=True)

def criterion(prediction, true_value):
    sm_pred = softmax(prediction)
    idx = range(len(prediction))
    return -torch.log(sm_pred[idx,true_value]).mean()

def get_lr(): return 0.01
def get_epoch(): return 20

model = SignNet()
optimizer = torch.optim.SGD(params=model.parameters(), lr=get_lr())

In [87]:
def train_model(model, optimizer):
    
    since = time.time()
    best_acc = 0.0
    for epoch in range(get_epoch()):
        
        print(f'epoch {epoch}/{get_epoch()-1}')
        print('=' * 15)
            
        # Each epoch has a training and validation phase
        for phase in ['train_loader', 'valid_loader']:
            if phase == 'train_loader':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in loadData()[phase]:
                #inputs = inputs.to(device)
                #labels = labels.to(device)
                
            

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train_loader'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
            
                        # backward
                    if phase == 'train_loader':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            #if phase == 'train_loader':
             #   scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase[:5]]
            epoch_acc = running_corrects.double() / dataset_sizes[phase[:5]]
            
            print(f'{phase[:5]} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # deep copy the model
            if phase == 'valid_loader' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
                

        print()
        
        
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [88]:
mod=train_model(model, optimizer)

epoch 0/19
train Loss: 2.3032 Acc: 0.1011
valid Loss: 2.3011 Acc: 0.1000

epoch 1/19
train Loss: 2.2998 Acc: 0.1232
valid Loss: 2.2961 Acc: 0.1567

epoch 2/19
train Loss: 2.2800 Acc: 0.1787
valid Loss: 2.2462 Acc: 0.2467

epoch 3/19
train Loss: 1.8959 Acc: 0.3458
valid Loss: 2.7986 Acc: 0.3200

epoch 4/19
train Loss: 1.1517 Acc: 0.6121
valid Loss: 1.7746 Acc: 0.4167

epoch 5/19
train Loss: 0.6997 Acc: 0.7763
valid Loss: 0.9173 Acc: 0.7067

epoch 6/19
train Loss: 0.4460 Acc: 0.8540
valid Loss: 0.4698 Acc: 0.8667

epoch 7/19
train Loss: 0.3176 Acc: 0.9042
valid Loss: 0.4734 Acc: 0.8367

epoch 8/19
train Loss: 0.2155 Acc: 0.9346
valid Loss: 0.5422 Acc: 0.8267

epoch 9/19
train Loss: 0.1702 Acc: 0.9527
valid Loss: 0.3075 Acc: 0.8933

epoch 10/19
train Loss: 0.1024 Acc: 0.9696
valid Loss: 0.2601 Acc: 0.9133

epoch 11/19
train Loss: 0.0969 Acc: 0.9708
valid Loss: 0.3528 Acc: 0.8900

epoch 12/19
train Loss: 0.0598 Acc: 0.9819
valid Loss: 0.3307 Acc: 0.9133

epoch 13/19
train Loss: 0.0365 Acc:

In [ ]:
def accuracy(prediction,label): 
    pred_sm = softmax(prediction).argmax(axis=1)
    count = label == pred_sm
    acc = count.float().mean()
    return acc

In [93]:
def export_model(dl_model, export_dir, exported_model="model.pth"):
    if not os.path.isdir(export_dir):
        export_path = os.mkdir(export_dir)
    model_path = path.Path(export_dir)/exported_model
    
    timestamp = datetime.datetime.now().strftime("-%Y-%m-%d-%H-%M-%S")
    model_path = path.Path(str(model_path)+timestamp)
    
    torch.save(dl_model.state_dict(), model_path)
    
def inference(img, model_path):
    model = SignNet()
    model.load_state_dict(torch.load(model_path))
    #with model.eval():
    infer_img = transform(PIL.Image.open(img))
    infer_img=infer_img.view(1, infer_img.shape[0], infer_img.shape[1], infer_img.shape[2])
    outputs = softmax(model(infer_img))
    probability, classes = torch.max(outputs, 1)
    
    return f'Prediction: {classes.item()}; Probability: {probability.item()}'

In [94]:
export_model(mod, "../outputs")

In [97]:
im_path = (path.Path("../data/Examples/example_4.JPG"))
os.path.isfile(im_path)

True

In [96]:
os.listdir("../outputs")[0]

'model.pth-2022-05-02-00-11-03'

In [100]:
inference(im_path, "../outputs/"+os.listdir("../outputs")[0])

'Prediction: 4; Probability: 0.9994024634361267'

In [98]:
im_path

Path('../data/Examples/example_4.JPG')